# RunnableParallel

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [3]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [4]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

In [5]:
string_parser = StrOutputParser()

In [6]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [7]:
# Splits the input to run chain_books and chain_projects in parallel, returning output as a dictionary with keys
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [8]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup.\n2. Developing a simple GUI-based application using Tkinter.\n3. Creating a basic Machine Learning model using Scikit-learn.'}

In [9]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [10]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: total: 0 ns
Wall time: 3.35 s


'1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin'

In [11]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 2.28 s


'1. Building a Weather Forecasting Application using APIs.\n2. Developing a Personal Finance Tracker with Data Visualization.\n3. Creating a Text-based Adventure Game with Multiple Scenarios.'

In [12]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: total: 31.2 ms
Wall time: 3.28 s


{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup\n2. Developing a simple Machine Learning application with Scikit-learn\n3. Creating a GUI-based Music Player using Tkinter'}